In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts

from sklearn.preprocessing import StandardScaler, MinMaxScaler  
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso        # regularizacion L1
from sklearn.linear_model import Ridge        # regularizacion L2
from sklearn.linear_model import ElasticNet

from sklearn.linear_model import LogisticRegression as LogReg

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from lightgbm import LGBMRegressor as LGBMR
from sklearn.model_selection import GridSearchCV
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR
from sklearn.naive_bayes import GaussianNB


In [46]:
def ml1():
    
    models = {'RFR':{'MODEL':RFR(),'PARAM':{'n_estimators': [10, 50, 100, 150, 200, 500],'max_depth': [1, 5, 10, 15, 20],'min_weight_fraction_leaf':[0.0,0.1,0.2]}},
              'XGB':{'MODEL':XGBR(),'PARAM':{'n_estimators': [10, 50, 100, 150, 200, 500],'max_depth': [1, 5,6, 10, 15, 20],'learning_rate':[0.001,0.002,0.01,0.05] }},
              #'CTR':{'MODEL':CTR(),'PARAM:':{'depth' : [6,8,10],'learning_rate' : [0.01, 0.05, 0.1],'iterations': [30, 50, 100],'subsample' : [0.5, 0.7, 1.0]}},
              #'GaussianNB':{'MODEL':GaussianNB(),'PARAM':{'var_smoothing':[1e-09]}},
              'Lasso':{'MODEL':Lasso(),'PARAM':{'alpha':[0.3,0.5,0.7,0.9,1.0],'max_iter':[800,1000,1200]}},
              'LGBMR': {'MODEL':LGBMR(),'PARAM':{'boosting_type':['gbdt','dart'],'n_estimators':[10,50,100,150,200,500],}}}
    
            
            
                     
    rmse = []
    name = []
    #b = []
    score = []
    dfmodels = pd.DataFrame()
    for m in models:
        x = models[m]["MODEL"]
        p = models[m]["PARAM"]
        y_pred=grid(x,p).predict(X_test)
        #best= grid(x,p).best_params_   
        #sco = grid(x,p).best_score_
        MSE = mse(y_test, y_pred, squared=False)
        #score.append(sco)
        #b.append(best)
        rmse.append(MSE)
        name.append(m)
    dfmodels['Modelo'] = name
    dfmodels['RMSE'] = rmse
    #dfmodels['Best_Parametres'] = b
    #dfmodels['bestsco'] = score                        
    dfmodels.sort_values("RMSE",ascending=True,inplace=True,ignore_index=True)
    print(f'model {dfmodels.Modelo[0]} rmse {dfmodels.RMSE[0]} ')
    return dfmodels

In [47]:
def grid(modelo, param):
    
    g=GridSearchCV(modelo, # modelo de sklearn
                   param,  # dictio de parametros
                   cv=5,   # nº de cortes del cross-validation
                   return_train_score=True, # error en entrenamiento para checkear
                   n_jobs=-1  # usa todos los nucleos disponibles
                  )

    g.fit(X_train, y_train)
    print('Acierto test: {:.2f}'.format(g.score(X_test, y_test)))
    print('Acierto train: {:.2f}'.format(g.score(X_train, y_train)))
    print('Mejores parametros: {}'.format(g.best_params_))
    print('Modelo: {}'.format(modelo))
    print('Mejor acierto cv: {:.2f}'.format(g.best_score_))


   

    return g.best_estimator_.fit(X_train, y_train)

In [12]:
salaries=pd.read_csv(r'data/salaries_data.csv')
testeo=pd.read_csv(r'data/testeo.csv')
muestra=pd.read_csv(r'data/muestra.csv')

In [13]:
average = {"US":54132,"FR":41379,"GR":17093,"LU": 75305,"SI":29460,"DE":45485,"IN":4766,"GB":40519,"PK":4379,"MD":6480,"JP":45000,"CA":40000,"AS":52626,"IE":40000,"AE":50000,"MX":20000,"VN":2400,"BE":44300,"KE":7608,"ES":36000,"CH":50000,"CL":27816,"CN":16716,"DK":40000,"RO":8360,"MY":18877,"PR":33000,"HU":17640,"PT": 31200,"BG":22263, "HN": 14000,"TR":8400,"AU": 52338,"UA":10000,"NG":9000,"BR":19000,"NL":50000,"HK":56000,"SG":60000,"IR":6000,"NZ":61000,"RS":16800,"JE":60000,"RU":18000,"EE":22000,"CZ":26000,"AT":45000,"BO":6000,"PL":23000,"IT":38000,"MT":30000,"DZ":7000,"HR":12000,"PH":13000,"IQ":5500,"CO":12000,"AR":20000,"TN":6000,"IL":42000}

In [14]:
jobs = {'Data Scientist':124000,'Data Engineer':111000,'Data Analyst':71599,'Data Architect':160000,'Machine Learning':138000,'Head of Data':170000,'ETL Developer':110000,'Data Specialist':160000,'3D Computer Vision Researcher':90000}

In [15]:
#https://www.edureka.co/blog/data-analyst-vs-data-engineer-vs-data-scientist/#:~:text=The%20typical%20salary%20of%20a,see%20much%20difference%20at%20first.
#https://www.glassdoor.com/Salaries/data-architect-salary-SRCH_KO0,14.htm

In [16]:
lst3 = []
lst4 = []
for i in jobs.keys():
    lst3.append(i)
    lst4.append(jobs[i])

In [17]:
lst1 = []
lst2 = []
for i in average.keys():
    lst1.append(i)
    lst2.append(average[i])

In [18]:
data1 = {"cc":lst1,"salary":lst2}

In [19]:
data2 = {"job":lst3,"salary":lst4}

In [20]:
salary = pd.DataFrame(data= data1)

In [21]:
salary2= pd.DataFrame(data= data2)

In [22]:
salary2

,job,salary
0,Data Scientist,124000
1,Data Engineer,111000
2,Data Analyst,71599
3,Data Architect,160000
4,Machine Learning,138000
5,Head of Data,170000
6,ETL Developer,110000
7,Data Specialist,160000
8,3D Computer Vision Researcher,90000


In [23]:
salaries

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,140250,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,135000,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,270000,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,26005,RO,0,US,L
...,...,...,...,...,...,...,...,...,...,...,...
495,2022,SE,FT,Data Engineer,100000,USD,100000,US,100,US,M
496,2021,EN,FT,Big Data Engineer,435000,INR,5882,IN,0,CH,L
497,2022,SE,FT,Data Architect,90700,USD,90700,CA,100,CA,M
498,2021,MI,FT,Data Engineer,60000,GBP,82528,GB,100,GB,L


In [24]:
for i in salaries.job_title:
    if 'Engineer' in i:
        salaries.job_title.replace({i: 'Data Engineer'}, inplace = True)
    elif 'Analy' in i:
        salaries.job_title.replace({i: 'Data Analyst'}, inplace = True)
    elif 'Architect' in i:
        salaries.job_title.replace({i: 'Data Architect'}, inplace = True)
    elif 'Scien' in i:
        salaries.job_title.replace({i: 'Data Scientist'}, inplace = True)
    elif 'Machine'in i:
        salaries.job_title.replace({i: 'Machine Learning'}, inplace = True)
        

In [25]:
for i,e in enumerate(salaries.employee_residence):
    f = salary[salary.cc==e].salary
    f = int(f)
    salaries.employee_residence[i]=f

C:\Users\pabli\anaconda3\envs\clase\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [26]:
for i,e in enumerate(salaries.company_location):
    f = salary[salary.cc==e].salary
    f = int(f)
    salaries.company_location[i]=f

C:\Users\pabli\anaconda3\envs\clase\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [27]:
for i,e in enumerate(salaries.job_title):
    try:
        f = salary2[salary2.job==e].salary
        f = int(f)
        salaries.job_title[i]=f
    except:
        print(e)

C:\Users\pabli\anaconda3\envs\clase\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [28]:
numeric1 = ['work_year', 'remote_ratio','employee_residence',"company_location",'job_title']

In [29]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

salaries[numeric1]=scaler.fit_transform(salaries[numeric1])

salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0.857874,SE,FT,0.080224,140250,USD,140250,0.653275,0.697464,0.586726,M
1,0.857874,SE,FT,0.080224,135000,USD,135000,0.653275,0.697464,0.586726,M
2,-0.606075,MI,FT,-1.738256,100000,USD,100000,0.653275,0.697464,0.586726,M
3,-0.606075,MI,CT,0.080224,270000,USD,270000,0.653275,0.697464,0.586726,L
4,-0.606075,MI,FT,0.080224,22000,EUR,26005,-2.260379,-1.775814,0.586726,L


In [30]:
from sklearn.preprocessing import LabelEncoder

for i in list(salaries.select_dtypes(exclude=["int64", 'float64']).columns):
    salaries[i] = LabelEncoder().fit_transform(salaries[i])
    
salaries

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0.857874,3,2,0.080224,140250,14,140250,0.653275,0.697464,0.586726,1
1,0.857874,3,2,0.080224,135000,14,135000,0.653275,0.697464,0.586726,1
2,-0.606075,2,2,-1.738256,100000,14,100000,0.653275,0.697464,0.586726,1
3,-0.606075,2,0,0.080224,270000,14,270000,0.653275,0.697464,0.586726,0
4,-0.606075,2,2,0.080224,22000,5,26005,-2.260379,-1.775814,0.586726,0
...,...,...,...,...,...,...,...,...,...,...,...
495,0.857874,3,2,0.080224,100000,14,100000,0.653275,0.697464,0.586726,1
496,-0.606075,0,2,0.080224,435000,8,5882,-2.489158,-1.775814,0.293828,0
497,0.857874,3,2,2.341728,90700,14,90700,-0.246309,0.697464,-0.415023,1
498,-0.606075,2,2,0.080224,60000,6,82528,-0.213271,0.697464,-0.378233,0


In [31]:
salaries.drop(columns=['salary','salary_currency'],inplace=True)

In [32]:
salaries.head()


,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0.857874,3,2,0.080224,140250,0.653275,0.697464,0.586726,1
1,0.857874,3,2,0.080224,135000,0.653275,0.697464,0.586726,1
2,-0.606075,2,2,-1.738256,100000,0.653275,0.697464,0.586726,1
3,-0.606075,2,0,0.080224,270000,0.653275,0.697464,0.586726,0
4,-0.606075,2,2,0.080224,26005,-2.260379,-1.775814,0.586726,0


In [33]:
X = salaries.drop('salary_in_usd', axis=1)

y = salaries.salary_in_usd

X.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size
0,0.857874,3,2,0.080224,0.653275,0.697464,0.586726,1
1,0.857874,3,2,0.080224,0.653275,0.697464,0.586726,1
2,-0.606075,2,2,-1.738256,0.653275,0.697464,0.586726,1
3,-0.606075,2,0,0.080224,0.653275,0.697464,0.586726,0
4,-0.606075,2,2,0.080224,-2.260379,-1.775814,0.586726,0


In [39]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(X, y, train_size=0.9, test_size=0.1, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((450, 8), (50, 8), (450,), (50,))

In [45]:
help(XGBR)

Help on class XGBRegressor in module xgboost.sklearn:

class XGBRegressor(XGBModel, sklearn.base.RegressorMixin)
 |  XGBRegressor(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'reg:squarederror', **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost regression.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of gradient boosted trees.  Equivalent to number of boosting
 |          rounds.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      max_leaves :
 |          Maximum number of leaves; 0 indicates no limit.
 |      max_bin :
 |          If using histogram-based algorithm, maximum number of bins per feature
 |      grow_policy :
 |          Tree growing policy. 0: favor splitting at nodes closest to the node, i.e. grow
 |          depth-wise. 1: favor splitting at nodes with highest loss change.
 | 

In [48]:
ml1()

Acierto test: 0.41
Acierto train: 0.63
Mejores parametros: {'max_depth': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200}
Modelo: RandomForestRegressor()
Mejor acierto cv: 0.45
Acierto test: 0.43
Acierto train: 0.54
Mejores parametros: {'learning_rate': 0.05, 'max_depth': 1, 'n_estimators': 500}
Modelo: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None,
        

,Modelo,RMSE
0,LGBMR,46767.149597
1,XGB,49523.099525
2,RFR,50910.042970
3,Lasso,51504.401493


In [49]:
lgbmr = LGBMR(boosting_type="dart",n_estimators=200)

In [50]:
lgbmr.fit(X_train,y_train)

LGBMRegressor(boosting_type='dart', n_estimators=200)

In [51]:
y_pred=lgbmr.predict(X_test)

In [52]:
MSE = mse(y_test, y_pred, squared=False)

In [53]:
MSE

46767.14959734202

In [54]:
testeo

,work_year,experience_level,employment_type,job_title,salary,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,2020,SE,FT,Machine Learning Scientist,260000,USD,JP,0,JP,S
1,2020,MI,FT,Lead Data Analyst,87000,USD,US,100,US,L
2,2020,MI,FT,Data Analyst,85000,USD,US,100,US,L
3,2020,MI,FT,Machine Learning Engineer,299000,CNY,CN,0,CN,M
4,2020,MI,FT,Product Data Analyst,450000,INR,IN,100,IN,L
...,...,...,...,...,...,...,...,...,...,...
102,2022,SE,FT,Data Analyst,170000,USD,US,100,US,M
103,2022,SE,FT,Data Architect,192564,USD,US,100,US,M
104,2022,SE,FT,Data Scientist,230000,USD,US,100,US,M
105,2022,EN,FT,Data Analyst,52000,USD,CA,0,CA,M


In [55]:
for i in testeo.job_title:
    if 'Engineer' in i:
        testeo.job_title.replace({i: 'Data Engineer'}, inplace = True)
    elif 'Analy' in i:
        testeo.job_title.replace({i: 'Data Analyst'}, inplace = True)
    elif 'Architect' in i:
        testeo.job_title.replace({i: 'Data Architect'}, inplace = True)
    elif 'Scien' in i:
        testeo.job_title.replace({i: 'Data Scientist'}, inplace = True)
    elif 'Machine'in i:
        testeo.job_title.replace({i: 'Machine Learning'}, inplace = True)

In [56]:
for i,e in enumerate(testeo.employee_residence):
    f = salary[salary.cc==e].salary
    f = int(f)
    testeo.employee_residence[i]=f

C:\Users\pabli\anaconda3\envs\clase\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [57]:
for i,e in enumerate(testeo.company_location):
    f = salary[salary.cc==e].salary
    f = int(f)
    testeo.company_location[i]=f

C:\Users\pabli\anaconda3\envs\clase\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [58]:
for i,e in enumerate(testeo.job_title):
    try:
        f = salary2[salary2.job==e].salary
        f = int(f)
        testeo.job_title[i]=f
    except:
        print(e)

C:\Users\pabli\anaconda3\envs\clase\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [59]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

testeo[numeric1]=scaler.fit_transform(testeo[numeric1])

testeo.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,-1.873027,SE,FT,0.670805,260000,USD,0.134821,-1.608265,0.074734,S
1,-1.873027,MI,FT,-1.725099,87000,USD,0.717412,0.798509,0.687445,L
2,-1.873027,MI,FT,-1.725099,85000,USD,0.717412,0.798509,0.687445,L
3,-1.873027,MI,FT,0.076413,299000,CNY,-1.669604,-1.608265,-1.822980,M
4,-1.873027,MI,FT,-1.725099,450000,INR,-2.431974,0.798509,-2.624765,L


In [60]:
from sklearn.preprocessing import LabelEncoder

for i in list(testeo.select_dtypes(exclude=["int64", 'float64']).columns):
    testeo[i] = LabelEncoder().fit_transform(testeo[i])
    
testeo.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,-1.873027,3,1,0.670805,260000,11,0.134821,-1.608265,0.074734,2
1,-1.873027,2,1,-1.725099,87000,11,0.717412,0.798509,0.687445,0
2,-1.873027,2,1,-1.725099,85000,11,0.717412,0.798509,0.687445,0
3,-1.873027,2,1,0.076413,299000,2,-1.669604,-1.608265,-1.822980,1
4,-1.873027,2,1,-1.725099,450000,7,-2.431974,0.798509,-2.624765,0


In [61]:
testeo.drop(columns=['salary','salary_currency'],inplace=True)

In [62]:
testeo


,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size
0,-1.873027,3,1,0.670805,0.134821,-1.608265,0.074734,2
1,-1.873027,2,1,-1.725099,0.717412,0.798509,0.687445,0
2,-1.873027,2,1,-1.725099,0.717412,0.798509,0.687445,0
3,-1.873027,2,1,0.076413,-1.669604,-1.608265,-1.822980,1
4,-1.873027,2,1,-1.725099,-2.431974,0.798509,-2.624765,0
...,...,...,...,...,...,...,...,...
102,0.872369,3,1,-1.725099,0.717412,0.798509,0.687445,1
103,0.872369,3,1,2.316815,0.717412,0.798509,0.687445,1
104,0.872369,3,1,0.670805,0.717412,0.798509,0.687445,1
105,0.872369,0,1,-1.725099,-0.184162,-1.608265,-0.260741,1


In [63]:
lgbmr.fit(X_train, y_train)

LGBMRegressor(boosting_type='dart', n_estimators=200)

In [64]:
y_pred1 = lgbmr.predict(testeo)

In [65]:
len(y_pred1)

107

In [66]:
testeo['salary_in_usd'] = list(y_pred1)

In [67]:
testeo.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size,salary_in_usd
0,-1.873027,3,1,0.670805,0.134821,-1.608265,0.074734,2,121991.052135
1,-1.873027,2,1,-1.725099,0.717412,0.798509,0.687445,0,155518.096942
2,-1.873027,2,1,-1.725099,0.717412,0.798509,0.687445,0,155518.096942
3,-1.873027,2,1,0.076413,-1.669604,-1.608265,-1.822980,1,25900.994005
4,-1.873027,2,1,-1.725099,-2.431974,0.798509,-2.624765,0,25966.839798


In [68]:
f= list(testeo.columns)[:8]

In [69]:
col = f

tiro2 = testeo.drop(col, axis = 1, inplace = False)
tiro2 = tiro2.reset_index()
tiro2 = tiro2.rename(columns = {'index':'id'})
tiro2.head()

,id,salary_in_usd
0,0,121991.052135
1,1,155518.096942
2,2,155518.096942
3,3,25900.994005
4,4,25966.839798


In [70]:
tiro2.to_csv(r'./tiro4.csv', index=False)

In [68]:
help(LGBMR)

Help on class LGBMRegressor in module lightgbm.sklearn:

class LGBMRegressor(sklearn.base.RegressorMixin, LGBMModel)
 |  LGBMRegressor(boosting_type: str = 'gbdt', num_leaves: int = 31, max_depth: int = -1, learning_rate: float = 0.1, n_estimators: int = 100, subsample_for_bin: int = 200000, objective: Union[str, Callable, NoneType] = None, class_weight: Union[Dict, str, NoneType] = None, min_split_gain: float = 0.0, min_child_weight: float = 0.001, min_child_samples: int = 20, subsample: float = 1.0, subsample_freq: int = 0, colsample_bytree: float = 1.0, reg_alpha: float = 0.0, reg_lambda: float = 0.0, random_state: Union[int, numpy.random.mtrand.RandomState, NoneType] = None, n_jobs: int = -1, silent: Union[bool, str] = 'warn', importance_type: str = 'split', **kwargs)
 |  
 |  LightGBM regressor.
 |  
 |  Method resolution order:
 |      LGBMRegressor
 |      sklearn.base.RegressorMixin
 |      LGBMModel
 |      sklearn.base.BaseEstimator
 |      builtins.object
 |  
 |  Methods de